# 5. Education and health Employemnt

### 5.1. Data Labeling

In [36]:
# --- 데이터 로드 ---
metadata = pd.read_excel("/Users/yujin.sophia.kim/Desktop/data/WDI/archive/WDI_Indicators_Metadata.xlsx")
df = pd.read_csv("/Users/yujin.sophia.kim/Desktop/data/WDI/archive/WDI_Indicators_MainData.csv")

In [37]:
# --- 선택 & 리네임 (CHE 추가, 콤마 고침) ---
df = df[[
    "Country Name", "Time",
    "Current health expenditure (% of GDP) [SH.XPD.CHEX.GD.ZS]",
    "Domestic general government health expenditure (% of GDP) [SH.XPD.GHED.GD.ZS]",
    "Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.BA.ZS]",
    "Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.LO.ZS]",
    "Educational attainment, at least completed post-secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.PO.ZS]",
    "Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative) [SE.PRM.CUAT.ZS]",
    "Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative) [SE.TER.CUAT.ST.ZS]"
]].copy()

df = df.rename(columns={
    "Current health expenditure (% of GDP) [SH.XPD.CHEX.GD.ZS]": "GDP_Total_HealthExp",
    "Domestic general government health expenditure (% of GDP) [SH.XPD.GHED.GD.ZS]": "GDP_GovHealthExp",
    "Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative) [SE.TER.CUAT.BA.ZS]": "EDU_Bachelors_25Plus_Total",
    "Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.LO.ZS]": "EDU_LowerSecondary_25Plus_Total",
    "Educational attainment, at least completed post-secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.PO.ZS]": "EDU_PostSecondary_25Plus_Total",
    "Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative) [SE.PRM.CUAT.ZS]": "EDU_Primary_25Plus_Total",
    "Educational attainment, at least completed short-cycle tertiary, population 25+, total (%) (cumulative) [SE.TER.CUAT.ST.ZS]": "EDU_ShortCycleTertiary_25Plus_Total"
})

# --- 숫자형 변환 ---
for col in ["GDP_Total_HealthExp","GDP_GovHealthExp",
            "EDU_Bachelors_25Plus_Total","EDU_LowerSecondary_25Plus_Total",
            "EDU_PostSecondary_25Plus_Total","EDU_Primary_25Plus_Total",
            "EDU_ShortCycleTertiary_25Plus_Total"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df["Time"] = pd.to_numeric(df["Time"], errors="coerce").astype("Int64")

# --- 정부보건지출의 CHE 대비 비중(%) ---
df["GovShare_CHE_pct"] = 100 * df["GDP_GovHealthExp"] / df["GDP_Total_HealthExp"]

# --- 산점도용 집계 (국가별 평균) ---
govshare_avg = df.groupby("Country Name", as_index=False)[[
    "GDP_Total_HealthExp","GDP_GovHealthExp",
    "EDU_Bachelors_25Plus_Total","EDU_LowerSecondary_25Plus_Total",
    "EDU_PostSecondary_25Plus_Total","EDU_Primary_25Plus_Total",
    "EDU_ShortCycleTertiary_25Plus_Total","GovShare_CHE_pct"
]].mean()

### 5.2. Regressions

In [38]:
# --- 1) EDU_Bachelors_25Plus_Total ---

country_avg = (
    df.groupby("Country Name")[["GDP_GovHealthExp", "EDU_Bachelors_25Plus_Total"]]
      .mean(numeric_only=True)
      .dropna()
      .reset_index()
)
corr_val = country_avg["GDP_GovHealthExp"].corr(country_avg["EDU_Bachelors_25Plus_Total"])
fig_scatter = px.scatter(
    country_avg,
    x="GDP_GovHealthExp",
    y="EDU_Bachelors_25Plus_Total",
    text="Country Name",         # <- 인덱스 배열 대신 컬럼명 사용
    trendline="ols",             # statsmodels 미설치면 이 줄 제거
    title=f"1. Government Health Expenditure VS At least Bachelor's or equivalent, Age over 25 (%) — r = {corr_val:.2f}",
    labels={
        "GDP_GovHealthExp": "Government Health Expenditure (% of GDP)",
        "EDU_Bachelors_25Plus_Total": "Bachelor's or equivalent (%)"
    }
)
fig_scatter.update_traces(textposition="top center")
fig_scatter.show()

# --- 2) EDU_LowerSecondary_25Plus_Total ----

country_avg = (
    df.groupby("Country Name")[["GDP_GovHealthExp", "EDU_LowerSecondary_25Plus_Total"]]
      .mean(numeric_only=True)
      .dropna()
      .reset_index()
)
corr_val = country_avg["GDP_GovHealthExp"].corr(country_avg["EDU_LowerSecondary_25Plus_Total"])
fig_scatter = px.scatter(
    country_avg,
    x="GDP_GovHealthExp",
    y="EDU_LowerSecondary_25Plus_Total",
    text="Country Name",         # <- 인덱스 배열 대신 컬럼명 사용
    trendline="ols",             # statsmodels 미설치면 이 줄 제거
    title=f"2. Government Health Expenditure VS At least completed lower secondary, Age over 25 (%) — r = {corr_val:.2f}",
    labels={
        "GDP_GovHealthExp": "Government Health Expenditure (% of GDP)",
        "EDU_LowerSecondary_25Plus_Total": "Lower secondary (%)"
    }
)
fig_scatter.update_traces(textposition="top center")
fig_scatter.show()

# --- 3) EDU_PostSecondary_25Plus_Total ---

country_avg = (
    df.groupby("Country Name")[["GDP_GovHealthExp", "EDU_PostSecondary_25Plus_Total"]]
      .mean(numeric_only=True)
      .dropna()
      .reset_index()
)
corr_val = country_avg["GDP_GovHealthExp"].corr(country_avg["EDU_PostSecondary_25Plus_Total"])
fig_scatter = px.scatter(
    country_avg,
    x="GDP_GovHealthExp",
    y="EDU_PostSecondary_25Plus_Total",
    text="Country Name",         # <- 인덱스 배열 대신 컬럼명 사용
    trendline="ols",             # statsmodels 미설치면 이 줄 제거
    title=f"3. Government Health Expenditure VS At least completed post-secondary, Age over 25 (%) — r = {corr_val:.2f}",
    labels={
        "GDP_GovHealthExp": "Government Health Expenditure (% of GDP)",
        "EDU_PostSecondary_25Plus_Total": "Post-secondary (%)"
    }
)
fig_scatter.update_traces(textposition="top center")
fig_scatter.show()

# --- 4) EDU_Primary_25Plus_Total ---

country_avg = (
    df.groupby("Country Name")[["GDP_GovHealthExp", "EDU_Primary_25Plus_Total"]]
      .mean(numeric_only=True)
      .dropna()
      .reset_index()
)
corr_val = country_avg["GDP_GovHealthExp"].corr(country_avg["EDU_Primary_25Plus_Total"])
fig_scatter = px.scatter(
    country_avg,
    x="GDP_GovHealthExp",
    y="EDU_Primary_25Plus_Total",
    text="Country Name",         # <- 인덱스 배열 대신 컬럼명 사용
    trendline="ols",             # statsmodels 미설치면 이 줄 제거
    title=f"4. Government Health Expenditure VS At least completed primary, Age over 25 (%) — r = {corr_val:.2f}",
    labels={
        "GDP_GovHealthExp": "Government Health Expenditure (% of GDP)",
        "EDU_Primary_25Plus_Total": "Primary (%)"
    }
)
fig_scatter.update_traces(textposition="top center")
fig_scatter.show()

# --- 5) EDU_ShortCycleTertiary_25Plus_Total ---

country_avg = (
    df.groupby("Country Name")[["GDP_GovHealthExp", "EDU_ShortCycleTertiary_25Plus_Total"]]
      .mean(numeric_only=True)
      .dropna()
      .reset_index()
)
corr_val = country_avg["GDP_GovHealthExp"].corr(country_avg["EDU_ShortCycleTertiary_25Plus_Total"])
fig_scatter = px.scatter(
    country_avg,
    x="GDP_GovHealthExp",
    y="EDU_ShortCycleTertiary_25Plus_Total",
    text="Country Name",         # <- 인덱스 배열 대신 컬럼명 사용
    trendline="ols",             # statsmodels 미설치면 이 줄 제거
    title=f"5. Government Health Expenditure VS At least completed short-cycle tertiary, Age over 25 (%) — r = {corr_val:.2f}",
    labels={
        "GDP_GovHealthExp": "Government Health Expenditure (% of GDP)",
        "EDU_ShortCycleTertiary_25Plus_Total": "Short-cycle tertiary (%)"
    }
)
fig_scatter.update_traces(textposition="top center")
fig_scatter.show()

### 5.2. Time Series Analysis

In [39]:
import numpy as np

# 누적(≥) 지표 약칭
BAp   = "EDU_Bachelors_25Plus_Total"         # ISCED 6+
SCtp  = "EDU_ShortCycleTertiary_25Plus_Total" # ISCED 5+
PSnp  = "EDU_PostSecondary_25Plus_Total"      # ISCED 4+
LSp   = "EDU_LowerSecondary_25Plus_Total"     # ISCED 2+
PRMp  = "EDU_Primary_25Plus_Total"            # ISCED 1+

# 1) BA+ (배타): 그대로 BA 이상
df["EXC_BAplus"] = df[BAp]

# 2) Short-cycle만 (ISCED 5만): ≥5 에서 ≥6을 뺌
df["EXC_ShortCycle_only"] = df[SCtp] - df[BAp]

# 3) Post-secondary 비(非)대학만 (ISCED 4만): ≥4 에서 ≥5를 뺌
df["EXC_PostSecondary_only"] = df[PSnp] - df[SCtp]

# 4) Lower+Upper secondary 묶음(ISCED 2~3): ≥2 에서 ≥4를 뺌
#   *Upper secondary(≥3) 지표가 없어서 2와 3을 분리하긴 어려움
df["EXC_Lower_or_UpperSec"] = df[LSp] - df[PSnp]

# 5) Primary only or below-secondary(ISCED 1만): ≥1 에서 ≥2를 뺌
df["EXC_Primary_only"] = df[PRMp] - df[LSp]

# 6) Below primary(ISCED 0 이하)
df["EXC_BelowPrimary"] = 100 - df[PRMp]


In [40]:
YR0, YR1 = 2011, 2021
df_ts = df[df["Time"].between(YR0, YR1)].copy()

# 사용할 지표 (존재하는 것만 자동 사용)
edu_labels = {
    "EDU_Bachelors_25Plus_Total":        "BA+ (25+, %)",
    "EDU_LowerSecondary_25Plus_Total":   "Lower Secondary+ (25+, %)",
    "EDU_PostSecondary_25Plus_Total":    "Post-secondary+ (25+, %)",
    "EDU_Primary_25Plus_Total":          "Primary+ (25+, %)",
    "EDU_ShortCycleTertiary_25Plus_Total":"Short-cycle Tertiary (25+, %)",
}
cols = [c for c in edu_labels if c in df_ts.columns]
assert len(cols) > 0, "교육 지표 컬럼이 없습니다."

# ----------------------------
# 1) 연도별 글로벌 평균/중앙값
# ----------------------------
yearly = (df_ts
          .groupby("Time")[cols]
          .agg(["mean","median","count"])
          .reset_index())

# 컬럼 평탄화: <지표>_<통계>
yearly.columns = ["Time"] + [f"{a}_{b}" for a,b in yearly.columns.tolist()[1:]]

# ==== Mean만 한 그래프에 ====
# (1) mean 컬럼 → 보기 좋은 라벨로 변환해서 long 형태로
mean_map = {f"{c}_mean": edu_labels[c] for c in cols if f"{c}_mean" in yearly.columns}
df_mean = yearly[["Time"] + list(mean_map.keys())].rename(columns=mean_map)
long_mean = df_mean.melt(id_vars="Time", var_name="Indicator", value_name="Mean")

# (2) 한 그래프에 모든 지표 mean 라인
fig = px.line(
    long_mean, x="Time", y="Mean", color="Indicator", markers=True,
    title=f"Global Means by Indicator (2011–2021)",
    labels={"Mean": "% of 25+ population", "Indicator": ""}
)
fig.update_layout(
    legend=dict(
        orientation="v",  # 세로
        xanchor="left",   # 오른쪽 바깥에 둘 거라 left로 고정
        x=1.02,           # 플롯의 오른쪽 밖
        yanchor="top",
        y=1
    ),
    margin=dict(r=180)    # 오른쪽 여백 늘리기 (필요에 따라 조절)
)

fig.update_yaxes(range=[0, 100])  # 퍼센트이면 유지, 아니면 지워도 OK
fig.show()

In [41]:
import numpy as np

# 누적(≥) 지표 약칭
BAp   = "EDU_Bachelors_25Plus_Total"         # ISCED 6+
SCtp  = "EDU_ShortCycleTertiary_25Plus_Total" # ISCED 5+
PSnp  = "EDU_PostSecondary_25Plus_Total"      # ISCED 4+
LSp   = "EDU_LowerSecondary_25Plus_Total"     # ISCED 2+
PRMp  = "EDU_Primary_25Plus_Total"            # ISCED 1+

# 1) BA+ (배타): 그대로 BA 이상
df["EXC_BAplus"] = df[BAp]

# 2) Short-cycle만 (ISCED 5만): ≥5 에서 ≥6을 뺌
df["EXC_ShortCycle_only"] = df[SCtp] - df[BAp]

# 3) Post-secondary 비(非)대학만 (ISCED 4만): ≥4 에서 ≥5를 뺌
df["EXC_PostSecondary_only"] = df[PSnp] - df[SCtp]

# 4) Lower+Upper secondary 묶음(ISCED 2~3): ≥2 에서 ≥4를 뺌
#   *Upper secondary(≥3) 지표가 없어서 2와 3을 분리하긴 어려움
df["EXC_Lower_or_UpperSec"] = df[LSp] - df[PSnp]

# 5) Primary only or below-secondary(ISCED 1만): ≥1 에서 ≥2를 뺌
df["EXC_Primary_only"] = df[PRMp] - df[LSp]

# 6) Below primary(ISCED 0 이하)
df["EXC_BelowPrimary"] = 100 - df[PRMp]


In [42]:
import numpy as np
import pandas as pd
import plotly.express as px

# ===== 0) 설정 =====
YR0, YR1 = 2011, 2021
# COUNTRY = None  # 전세계 평균 (Global mean)
COUNTRY = None     # 한 나라를 보고 싶으면 예: "Korea, Rep."

# 배타 카테고리(있는 것만 자동 사용)
exc_cols_all = [
    "EXC_BAplus",
    "EXC_ShortCycle_only",
    "EXC_PostSecondary_only",
    "EXC_Lower_or_UpperSec",
    "EXC_Primary_only",
    "EXC_BelowPrimary",
]
exc_cols = [c for c in exc_cols_all if c in df.columns and c != "Time"]

# 숫자형/클리핑
for c in exc_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").clip(lower=0, upper=100)

df_ts = df[df["Time"].between(YR0, YR1)].copy()

# ===== 1) 데이터 집계: Global mean 또는 특정 국가 =====
if COUNTRY is None:
    # 전세계 평균(국가 가중치 없이 단순 평균)
    agg = (df_ts.groupby("Time")[exc_cols]
             .mean()
             .reset_index())
    title_prefix = "Global Composition (mean across countries)"
else:
    agg = (df_ts[df_ts["Country Name"] == COUNTRY]
             .sort_values("Time")[["Time"] + exc_cols]
             .drop_duplicates(subset=["Time"]))
    title_prefix = f"{COUNTRY} — Composition"

# (데이터 품질용) 각 연도 합 확인: 보통 ~100 근처여야 정상
agg["sum_check"] = agg[exc_cols].sum(axis=1)

# ===== 2) Long 변환 & 스택 면적그래프 =====
long = agg.melt(id_vars=["Time"], value_vars=exc_cols,
                var_name="Category", value_name="Percent")

# 보기 좋은 순서(아래→위): 학력 낮은→높은
order = [c for c in [
    "EXC_BelowPrimary",
    "EXC_Primary_only",
    "EXC_Lower_or_UpperSec",
    "EXC_PostSecondary_only",
    "EXC_ShortCycle_only",
    "EXC_BAplus",
] if c in exc_cols]

pretty = {
    "EXC_BelowPrimary":        "Below Primary (ISCED 0-)",
    "EXC_Primary_only":        "Primary only (ISCED 1)",
    "EXC_Lower_or_UpperSec":   "Lower/Upper Secondary (ISCED 2–3)",
    "EXC_PostSecondary_only":  "Post-secondary only (ISCED 4)",
    "EXC_ShortCycle_only":     "Short-cycle Tertiary only (ISCED 5)",
    "EXC_BAplus":              "BA+ (ISCED 6+)",
}

long["Category"] = long["Category"].map(pretty)

fig = px.area(
    long.sort_values(["Category","Time"]),
    x="Time", y="Percent", color="Category",
    category_orders={"Category": [pretty[k] for k in order]},
    title=f"{title_prefix}: Education Attainment Composition (25+), {YR0}–{YR1}",
    labels={"Percent":"Share of 25+ population (%)", "Time":"Year", "Category":""}
)
fig.update_layout(legend_title_text="")
fig.show()

# (선택) 합계 확인용 라인 플롯 — 각 연도 합이 100%에 얼마나 가까운지
# px.line(agg, x="Time", y="sum_check", markers=True,
#         title="Sum of exclusive shares per year (should be ~100%)",
#         labels={"sum_check":"Sum of shares (%)"}).show()
